# Assemble

This notebook loads and assembles all your basic data sources, including tabulor and geospatial data.

The final output is two dataframes:

- UNIVERSE
- SALES

The SALES dataframe represents transactions or parcels; these are ownership transfers with prices, dates, and metadata.  
The UNIVERSE dataframe represents the parcels themselves (land and buildings, and their associated characteristics).

These will be packaged together in a handy data structure called a `SalesUniversePair`, or `sup` for short. `openavmkit` provides many handy functions that carefully perform operations on `sup`s without mixing up their fields.

The key thing to understand is that the **Assemble** notebook outputs a `sup` that represents *factual assertions* about the world. In later notebooks, we will have to add assumptions, opinions, and educated guesses, but we first will establish the firmest facts we can in this notebook.

You can think of the two dataframes in the `sup` as answering the following questions:

- UNIVERSE:
  - Where is each parcel located in space, and what is its shape?
  - What are the *current* characteristics of each parcel?
    - Which parcels have buildings and which are vacant lots?
    - How big is each parcel?
    - What is the age/size/quality/condition/etc of each building?
- SALES:
  - Which parcels have sold?
  - What prices did they sell for?
  - What dates did they sell on?
  - Which sales were valid?
  - What characteristics were different *at the time of sale* from how the parcel is now?



In [1]:
# Change these as desired

# The slug of the locality you are currently working on
locality = "us-md-baltimorecity"

# Whether to print out a lot of stuff (can help with debugging) or stay mostly quiet
verbose = True

# Clear previous state for this notebook and start fresh
clear_checkpoints = True

# 1. Basic setup

In [2]:
import init_notebooks
init_notebooks.setup_environment()

Environment setup completed.


In [3]:
# import OpenAVMkit:
from openavmkit.pipeline import ( 
    init_notebook,
    from_checkpoint,
    delete_checkpoints,
    examine_df,
    examine_df_in_ridiculous_detail,
    examine_sup,
    examine_sup_in_ridiculous_detail,
    cloud_sync,
    load_settings,
    load_dataframes,
    process_data,
    process_sales,
    tag_model_groups_sup,
    write_notebook_output_sup
)

In [4]:
init_notebook(locality)

locality = us-md-baltimorecity
base path = /Users/gregmiller/Documents/openavmkit/openavmkit/notebooks/pipeline
current path = /Users/gregmiller/Documents/openavmkit/openavmkit/notebooks/pipeline/data/us-md-baltimorecity


In [5]:
if clear_checkpoints:
    delete_checkpoints("1-assemble")

In [6]:
settings = load_settings()

# 2. Sync with Cloud
- If you have configured cloud storage, syncs with your remote storage
- Reconciles your local input files with the versions on the remote server
- Pulls down whatever is newer from the remote server
- Uploads whatever is newer from your local machine

In [7]:
ignore_paths = [
    "in/aw/bright",
    "us/md/baltimorecity/raw/bright",
    "us/md/baltimorecity/aw",
    "us/md/baltimorecity/w/",
    "us/md/baltimorecity/aw/bright",  # Ignore the entire bright folder
    "us/md/baltimorecity/aw/bright/bd_20250303_173558.4.csv"  # Ignore a specific file
]
cloud_sync(locality, verbose=True, env_path="../../../.env", settings=settings, ignore_paths=ignore_paths)

# 3. Load & process data

In [8]:
# load all of our initial dataframes, but don't do anything with them just yet
dataframes = from_checkpoint("1-assemble-01-load_dataframes", load_dataframes,
    {
        "settings":settings,
        "verbose":verbose
    }
)

Loading "in/geo/regrid_parcels.parquet"...
Loading "in/Real_Property_Information.parquet"...
Loading "in/bright_combined_reduced.parquet"...
Loading "in/sales.parquet"...
Valid sales: 101546 out of 241196 total
Loading "in/sales.parquet"...
Valid sales: 58477 out of 241196 total
Loading "in/sales.parquet"...
Valid sales: 39235 out of 241196 total


FileNotFoundError: [Errno 2] No such file or directory: 'in/likely_garage.csv'

In [ ]:
# assemble our data
sales_univ_pair = from_checkpoint("1-assemble-02-process_data", process_data,
    {
        "dataframes":dataframes, 
        "settings":settings, 
        "verbose":verbose
    }
)

Dropped 2375661 duplicate rows based on '['key']'
Valid sales: 199248 (27.5% of 723528 total)
Enriching universe...
Performing basic geometric enrichment...
--> added latitude/longitude...(1.51s)
--> calculated GIS area of each parcel...(0.03s)
HO
YO
--> calculated parcel rectangularity...(27.59s)
--> calculated parcel aspect ratios...(0.73s)
--> identifying irregular parcels...
----> simplified geometry...(1.89s)
----> identified triangular parcels...(25.62s)
----> identified complex geometry...(1.83s)
----> identified elongated parcels...(0.00s)
----> finished up...(0.70s)
--> identified irregular parcels (total)...(30.04s)
--> calculated polar coordinates...(9.90s)
Performing spatial joins...
Enriching with Overture building data...
--> Current settings: {'enabled': True, 'cache': True, 'footprint': {'units': 'sqft'}}
--> Bounding box: [-76.711309   39.1972805 -76.5296665  39.372015 ]
--> Fetching data from Overture...
--> Counting batches...
--> Found 243 batches


Processing batches: 100%|██████████| 243/243 [00:14<00:00, 17.01it/s]


--> Found 280484 buildings
--> Available columns: ['id', 'geometry', 'bbox', 'theme', 'type', 'version', 'sources', 'level', 'subtype', 'class', 'height', 'names', 'has_parts', 'is_underground', 'num_floors', 'num_floors_underground', 'min_height', 'min_floor', 'facade_color', 'facade_material', 'roof_material', 'roof_shape', 'roof_direction', 'roof_orientation', 'roof_color', 'roof_height']
--> Calculating building footprint areas...
--> Using UTM CRS: EPSG:32618
--> Saving buildings to cache: cache/overture/buildings_-76.711309_39.1972805_-76.5296665_39.372015.parquet
--> Projected to equal area CRS...(1.02s)
--> Calculated building footprint intersections with parcels...(3.68s)
--> Found 1575916 potential building-parcel intersections
--> Calculated precise intersection areas...(69.54s)
--> Aggregated building footprint areas...(0.13s)
--> Finished up...(0.23s)
--> Added building footprint areas to 236505 parcels
--> Total building footprint area: 1,544,685,360 sqft
--> Average buil

In [ ]:
# process the sales
sales_univ_pair = from_checkpoint("1-assemble-03-process_sales", process_sales,
    {
        "sup": sales_univ_pair, 
        "settings": settings, 
        "verbose": verbose
    }
)

Before univ merge len = 197821
After univ merge len = 197821
Using 53837 sales...
--> 512 vacant sales
--> 53325 improved sales
--> 53205 valid for ratio study
--> 512 valid for land ratio study
len before hydrate = 53837
len after hydrate = 53837
Applying time adjustment...
Calculating time adjustment...


/Users/gregmiller/Documents/openavmkit/openavmkit/.venv/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/gregmiller/Documents/openavmkit/openavmkit/.venv/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


--> Using period: Q
--> Crunching time adjustment...
--> Flattening time adjustment...
---->interpolating missing periods...
--> Time adjustment calculated for 5183 days.
len after enrich = 53837
Number of parcels in universe with coordinates: 236505 / 236505
Number of parcels in universe with coordinates: 236505 / 236505
Number of parcels in universe with coordinates: 236505 / 236505
Number of parcels in universe with coordinates: 236505 / 236505


# 4. Inspect results

## 4.1 Examine

- Run the next cell and look at the printed out results.
- Note the "Non-zero" and "Non-null" columns in particular and make sure they're what you expect
- This view is for a quick glance to get a good idea of what all your data is

In [ ]:
settings = load_settings()
#examine_sup(sales_univ_pair, settings)


EXAMINING UNIVERSE...

            FIELD                 TYPE     NON-ZERO    %    NON-NULL    %                    UNIQUE                 
============================== ========== ========== ===== ========== ===== ========================================
             LAND             
============================== ========== ========== ===== ========== ===== ========================================
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
           NUMERIC            
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
assr_land_value                 Float64      219,315   93%    236,505  100%                                         
exempt_land_value               Float64       16,452    7%    236,471  100%                                         
geom_aspect_ratio               float64      236,505  100%    236,505  100%                    

## 4.2 Examine in ridiculous detail

- You've looked, now LOOK AGAIN. This cell will run `describe()` for each numeric field and `value_counts()` for each categorical field.
- Use this info to decide which variables are useful/useless
- Consult this readout when you build your modeling group filters

In [ ]:
#examine_sup_in_ridiculous_detail(sales_univ_pair, settings)


EXAMINING UNIVERSE...

            FIELD                 TYPE     NON-ZERO    %    NON-NULL    %                    UNIQUE                 
============================== ========== ========== ===== ========== ===== ========================================
             LAND             
============================== ========== ========== ===== ========== ===== ========================================
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
           NUMERIC            
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
assr_land_value                 Float64      219,315   93%    236,505  100%                                         
DESCRIBE --> count        236505.0
mean     52546.581189
std      348412.83972
min               0.0
25%           10000.0
50%           30000.0
75%           57000.0
max       123000000.0
Name: assr_land_value, 

## 4.3 Look at it on a map

- Go to your `out/look/` folder
- There should be parquets there
- Drop them into ArcGIS, QGIS, or Felt
- Look at your location fields and make sure they make sense

# 5. Tag modeling groups
- Separates rows into groups like "single family", "townhomes" and "commercial" as specified by the user
- These groups will guide all further processing

In [ ]:
settings = load_settings()
sales_univ_pair = from_checkpoint("1-assemble-04-tag_modeling_groups", tag_model_groups_sup,
    {
        "sup": sales_univ_pair, 
        "settings": settings, 
        "verbose": verbose
    }
)

Len univ before = 236505
Len sales before = 53837 after = 53837
Overall
--> 236,505 parcels
--> 53,837 sales
Assigning model group condos...
common_area -->  False
Assigning model group apartments...
common_area -->  False
Assigning model group industrial...
common_area -->  False
Assigning model group rowhome...
common_area -->  False
Assigning model group single_family...
common_area -->  False
Assigning model group commercial...
common_area -->  False
 --> 0 parcels had their model group changed.
Condos
--> 12,869 parcels
--> 2,979 sales
----> 2,974 improved sales
----> 5 vacant sales
Apartments
--> 3,713 parcels
--> 374 sales
----> 359 improved sales
----> 15 vacant sales
Industrial
--> 4,318 parcels
--> 509 sales
----> 483 improved sales
----> 26 vacant sales
Rowhome/townhouse
--> 167,977 parcels
--> 40,788 sales
----> 40,537 improved sales
----> 251 vacant sales
Single-family
--> 39,122 parcels
--> 8,722 sales
----> 8,585 improved sales
----> 137 vacant sales
Commercial
--> 24,04

# 6. Write out results

In [14]:
write_notebook_output_sup(sales_univ_pair, "1-assemble")

Results written to:
...out/1-assemble-sup.pickle
...out/look/1-assemble-universe.parquet
...out/look/1-assemble-sales.parquet
...out/look/1-assemble-sales-hydrated.parquet


# 7. Look at it on a map!
- Take the files output in the previous step and put them in a map viewer like QGIS, ArcGIS, or Felt
- Look at them with your eyeballs
- Make sure the data looks correct
- If not, go back and fix it!
- Don't proceed to the next step until everything looks right

In [ ]:
# Let's examine the sales and universe data
def analyze_data_quality(sales_univ_pair):
   
    sales_df = sales_univ_pair["sales"]
    universe_df = sales_univ_pair["universe"]
    # Print head of top ten assr_bldg_area_finished_sqft
    print("\n=== Top 10 Properties by Building Area ===")
    if 'assr_bldg_area_finished_sqft' in universe_df.columns:
        # Sort by building area in descending order and get top 10
        top_area_properties = universe_df.sort_values(by='assr_bldg_area_finished_sqft', ascending=False).head(10)
        
        # Select relevant columns to display
        columns_to_display = ['key', 'assr_bldg_area_finished_sqft', 'address', 'model_group']
        display_columns = [col for col in columns_to_display if col in top_area_properties.columns]
        
        print(f"\nTop 10 properties by assr_bldg_area_finished_sqft:")
        print(top_area_properties[display_columns])
    else:
        print("Column 'assr_bldg_area_finished_sqft' not found in the dataset")

    print("=== B&D Sales Check ===")
    bd_sales = sales_df[sales_df['sale_grantor'].str.contains('B&D', na=False, case=False)]
    if len(bd_sales) > 0:
        print(f"\nFound {len(bd_sales)} B&D sales:")
        print(bd_sales[['key', 'sale_grantor', 'sale_price', 'sale_date', 'convey_type']].head())
    else:
        print("No B&D sales found (good!)")
        
    print("\n=== Specific Parcel Price Tweak Check ===")
    # Check some of the specific parcel tweaks from settings.json
    parcels_to_check = [
        "1654043---2023-05-01",
        "1846082---2024-12-02",
        "6434025H---2023-02-08"
    ]
    tweaked_sales = sales_df[sales_df['key_sale'].isin(parcels_to_check)]
    if len(tweaked_sales) > 0:
        print("\nFound tweaked parcels:")
        print(tweaked_sales[['key_sale', 'sale_price', 'sale_date']].head())
    else:
        print("No tweaked parcels found")
        
    print("\n=== Rowhome Analysis ===")
    # Check for Commercial (C) use codes in rowhome zoning
    rowhome_commercial = universe_df[
        (universe_df['model_group'] == 'rowhome') & 
        (universe_df['parcel_use_code'] == 'C')
    ]
    print(f"\nRowhomes with Commercial use code: {len(rowhome_commercial)}")
    if len(rowhome_commercial) > 0:
        print("\nSample of rowhomes with commercial use:")
        print(rowhome_commercial[['key', 'parcel_use_code', 'zoning_desc', 'model_group']].head())
    
    print("\n=== Model Group Distribution ===")
    print("\nUniverse model group counts:")
    print(universe_df['model_group'].value_counts())
    
    print("\nSales model group counts:")
    sales_with_model_group = sales_df.merge(
        universe_df[['key', 'model_group']], 
        on='key', 
        how='left'
    )
    print(sales_with_model_group['model_group'].value_counts())

# Run the analysis
analyze_data_quality(sales_univ_pair)


=== Top 10 Properties by Building Area ===

Top 10 properties by assr_bldg_area_finished_sqft:
              key  assr_bldg_area_finished_sqft               address  \
63478     1807001                       32760.0     1429 ALICEANNA ST   
95040     2846006                       32760.0    2912 GARRISON BLVD   
227980  7900F001A                       32722.0          903 COOKS LN   
40029     1236001                       32588.0     700 E MONUMENT ST   
228389   7993001B                       32506.0  701 N CHAPEL GATE LN   
198112    6065013                       32493.0      4801 LORELLY AVE   
94470     2812011                       32484.0    2200 GARRISON BLVD   
32569     0969004                       32472.0       611 W OSTEND ST   
32076     0965001                       32468.0     1201 S CHARLES ST   
117414   3642D003                       32440.0        2606 SISSON ST   

       model_group  
63478   industrial  
95040   apartments  
227980  apartments  
40029   industri

In [16]:
# Find sales with both "CITY" and "MAYOR" in the grantor name
sales_df = sales_univ_pair["sales"]
city_mayor_sales = sales_df[
    sales_df['sale_grantor'].str.contains('CITY', na=False, case=False) & 
    sales_df['sale_grantor'].str.contains('MAYOR', na=False, case=False)
]

print("\n=== Sales with 'CITY' and 'MAYOR' in Grantor Name ===")
if len(city_mayor_sales) > 0:
    print(f"Found {len(city_mayor_sales)} sales with both 'CITY' and 'MAYOR' in grantor name:")
    
    # Display sample of these sales
    print("\nSample of sales with 'CITY' and 'MAYOR' in grantor name:")
    print(city_mayor_sales[['key', 'sale_grantor', 'sale_price', 'sale_date', 'convey_type']].head(10))
    
    # Group by grantor name and sale price, then count occurrences
    print("\nCounts of each sale price by grantor name containing 'CITY' and 'MAYOR':")
    grantor_price_counts = city_mayor_sales.groupby(['sale_grantor', 'sale_price']).size().reset_index(name='count')
    
    # Sort by grantor name and then by count (descending)
    grantor_price_counts = grantor_price_counts.sort_values(['sale_grantor', 'count'], ascending=[True, False])
    
    # Display the counts
    for grantor in grantor_price_counts['sale_grantor'].unique():
        print(f"\nGrantor: {grantor}")
        grantor_data = grantor_price_counts[grantor_price_counts['sale_grantor'] == grantor]
        for _, row in grantor_data.iterrows():
            print(f"  Sale price: ${row['sale_price']:,.2f} - Count: {row['count']}")
else:
    print("No sales with both 'CITY' and 'MAYOR' in grantor name found")



=== Sales with 'CITY' and 'MAYOR' in Grantor Name ===
No sales with both 'CITY' and 'MAYOR' in grantor name found


In [17]:
# Find sales with both "HOUSING" and "AUTH" in the grantor name
sales_df = sales_univ_pair["sales"]
housing_authority_sales = sales_df[
    sales_df['sale_grantor'].str.contains('HOUSING', na=False, case=False) & 
    sales_df['sale_grantor'].str.contains('AUTH', na=False, case=False)
]

print("\n=== Sales with 'HOUSING' and 'AUTH' in Grantor Name ===")
if len(housing_authority_sales) > 0:
    print(f"Found {len(housing_authority_sales)} sales with both 'HOUSING' and 'AUTH' in grantor name:")
    
    # Display sample of these sales
    print("\nSample of sales with 'HOUSING' and 'AUTH' in grantor name:")
    print(housing_authority_sales[['key', 'sale_grantor', 'sale_price', 'sale_date', 'convey_type']].head(10))
    
    # Group by grantor name and sale price, then count occurrences
    print("\nCounts of each sale price by grantor name containing 'HOUSING' and 'AUTH':")
    grantor_price_counts = housing_authority_sales.groupby(['sale_grantor', 'sale_price']).size().reset_index(name='count')
    
    # Sort by grantor name and then by count (descending)
    grantor_price_counts = grantor_price_counts.sort_values(['sale_grantor', 'count'], ascending=[True, False])
    
    # Display the counts
    for grantor in grantor_price_counts['sale_grantor'].unique():
        print(f"\nGrantor: {grantor}")
        grantor_data = grantor_price_counts[grantor_price_counts['sale_grantor'] == grantor]
        for _, row in grantor_data.iterrows():
            print(f"  Sale price: ${row['sale_price']:,.2f} - Count: {row['count']}")
else:
    print("No sales with both 'HOUSING' and 'AUTH' in grantor name found")



=== Sales with 'HOUSING' and 'AUTH' in Grantor Name ===
No sales with both 'HOUSING' and 'AUTH' in grantor name found



=== Top 10 Properties by Building Area ===


NameError: name 'universe_df' is not defined